In [2]:
import numpy as np

In [3]:
filenames = ['data/189_2011.txt', 'data/189_2012.txt', 'data/189_2013.txt', 'data/189_2014.txt',
             'data/189_2015.txt', 'data/189_2016.txt', 'data/189_2017.txt', 'data/189_2018.txt',
             'data/189_2019.txt', 'data/189_2020.txt', 'data/189_2021.txt']
raw_data = []
for f in filenames:
    raw_data = np.concatenate((raw_data, np.loadtxt(f, delimiter=',', skiprows=1, dtype=str)))

In [ ]:
data = np.genfromtxt((i[:-2] if i[-1] == '4' else '' for i in raw_data), skip_header=14, delimiter=';', 
                     dtype=[('date', 'U13'), ('height', 'float')])

In [ ]:
data

In [4]:
raw_data

array(['07/09/2011 08:57:20;3.64;1', '07/09/2011 08:58:20;4.19;1',
       '07/09/2011 08:59:20;4.225;1', ..., '07/09/2021 12:45:30;4.3251;1',
       '07/09/2021 12:46:30;4.3591;1', '07/09/2021 12:47:30;4.3694;1'],
      dtype='<U32')